In [1]:
import pandas as pd

In [8]:
cuadros = ["C-4", "C 7", "C 8", "C 9"]

In [33]:
data = pd.read_excel("./data/anuario_pecuaria_2024.xlsx",
                     sheet_name=cuadros[0]
                     )

In [34]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"C.4 PERÚ: POBLACIÓN PECUARIA POR ESPECIE, SEG...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(Unidades),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Región,Ave,Alpaca,LLama,Caprino,Ovino,Porcino,Vacuno,Vacas en Ordeño
4,Total Nacional,194758710,4545820,1056907,1749263,11278083,3519668,5842166,933906.990558


In [35]:
starts = data.index[data.apply(lambda r: r.astype(str).str.contains("POBLACIÓN PECUARIA POR ESPECIE, SEGÚN REGIÓN").any(), axis=1)]

In [36]:
starts

Index([0], dtype='int64')